In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
import warnings
warnings.filterwarnings("ignore")

In [3]:
df1 = pd.read_csv(r"C:\Users\yuvra\Downloads\Products.csv")

In [4]:
df2 = pd.read_csv(r"C:\Users\yuvra\Downloads\Sales.csv.csv")

FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\yuvra\\Downloads\\Sales.csv.csv'

In [ ]:
df3 = pd.read_csv(r"C:\Users\yuvra\Downloads\Inventory.csv.csv")

In [ ]:
df2['Date']= pd.to_datetime(df2['Date'])

In [ ]:
merge_df = pd.merge(df2,df1 , on='ProductId')

In [ ]:
merge_df

In [ ]:
sales_trends = merge_df.groupby('Date').agg({'UnitPrice': 'sum', 'Quantity': 'sum'})

In [ ]:
sales_trends

In [ ]:
# Grouping by Date to analyze sales trends
top_selling_products = merge_df.groupby("ProductName")['Quantity'].sum().nlargest(5)

In [ ]:
top_selling_products

In [ ]:
sales_by_month = merge_df.groupby(merge_df['Date'].dt.month)['Quantity'].sum()

In [ ]:
sales_by_month

In [ ]:
print("Sales trends over time:\n", sales_trends.head())
print('-----------------------------------------------')
print("\nTop selling products:\n", top_selling_products)
print('-----------------------------------------------')
print("\nMonthly sales analysis:\n", sales_by_month)

In [ ]:

# Top selling products
plt.figure(figsize=(8, 5))
top_selling_products.plot(kind='bar')
plt.title('Top Selling Products')
plt.xlabel('Product')
plt.ylabel('Total Quantity Sold')
plt.show()

# Monthly sales analysis
plt.figure(figsize=(8, 5))
sns.barplot(x=sales_by_month.index, y=sales_by_month.values)
plt.title('Monthly Sales Analysis')
plt.xlabel('Month')
plt.ylabel('Total Quantity Sold')
plt.show()

In [ ]:
X = merge_df[['UnitPrice', 'ProductCost']]
y = merge_df['Quantity']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = LinearRegression()
model.fit(X_train, y_train)

predictions = model.predict(X_test)

In [ ]:
# Visualizing predicted vs actual sales quantities using a regression plot
plt.figure(figsize=(12, 8))
sns.regplot(x=y_test, y=predictions, scatter_kws={'alpha':0.5})
plt.title('Predicted vs Actual Sales Quantities')
plt.xlabel('Actual Quantity Sold')
plt.ylabel('Predicted Quantity Sold')
plt.ylim(50, 50.35)
plt.show()

In [ ]:
predictions

In [ ]:

# Calculating residuals
residuals = y_test - predictions

# Creating a residual plot
plt.figure(figsize=(8, 6))
sns.residplot(x=predictions, y=residuals, lowess=True, scatter_kws={'alpha': 0.5})
plt.title('Residual Plot')
plt.xlabel('Predicted Quantity Sold')
plt.ylabel('Residuals')
plt.xlim(50, 50.35)
plt.axhline(y=0, color='red', linestyle='--')  # Adding a horizontal line at y=0 for reference
plt.show()

In [ ]:
plt.figure(figsize=(8, 6))
sns.histplot(residuals, kde=True, bins=20)
plt.title('Distribution of Residuals')
plt.xlabel('Residuals')
plt.ylabel('Frequency')
plt.axvline(x=0, color='red', linestyle='--')  # Adding a vertical line at x=0 for reference
plt.show()

In [ ]:
# Analyzing profitability by product
merge_df['Profit'] = merge_df['UnitPrice'] - merge_df['ProductCost']
profitable_products = merge_df.groupby('ProductName')['Profit'].mean().nlargest(10)

profitable_products

In [ ]:
merge_df['Profit'] = merge_df['UnitPrice'] - merge_df['ProductCost']
profitable_products = merge_df.groupby('ProductName')['Profit'].mean().nlargest(10)

profitable_products

In [ ]:
plt.figure(figsize=(10,6))
sns.barplot(x=profitable_products.index, y =profitable_products.values)
plt.title('top 10 Profitable Products')
plt.xlabel('Product')
plt.ylabel('Mean Profit')
plt.xticks(rotation=60)
plt.show()

In [ ]:
top_products = merge_df.groupby('ProductName')['Quantity'].sum().nlargest(5)


In [ ]:
top_products

In [ ]:
top_products_df = merge_df[merge_df['ProductName'].isin(top_products.index)]
top_products_df

In [ ]:
plt.figure(figsize=(10,6))
sns.scatterplot(data=top_products_df , x='Quantity',y='Profit', hue = 'ProductName',palette='Set1')
plt.title('Profit per unit vs Quantity Sold for Top Products')
plt.xlabel('Total Quantity Sold')
plt.ylabel('Profit')
plt.legend(title='Product')
plt.show()

In [ ]:
# Creating bins for profit ranges
profit_bins = np.linspace(merge_df['Profit'].min(), merge_df['Profit'].max(), 10)
profit_bins

In [ ]:
merge_df['Profit_Bin']=pd.cut(merge_df['Profit'],bins=profit_bins,labels=False)

In [ ]:
profit_range_totals = merge_df.groupby('Profit_Bin')['Profit'].sum()

In [ ]:
profit_percentages =(profit_range_totals / profit_range_totals.sum())*100

In [ ]:
# Creating a pie chart to show profit percentages for each range of products
plt.figure(figsize=(8, 8))
plt.pie(profit_percentages, labels=profit_percentages.index, autopct='%1.1f%%', startangle=140)
plt.title('Profit Percentage for Each Range of Products')
plt.axis('equal')  # Equal aspect ratio ensures that pie is drawn as a circle.
plt.show()